In [7]:
import pandas as pd
import numpy as np
from operator import itemgetter

gene_coords_file = '/data/EBV/by_gene/gene_result.txt'
repeats_analysis_file = '/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/rdp4_results/ids_171_ebv/10Miter/results/repeats_analysis.csv'
dfrep = pd.read_csv(repeats_analysis_file)

gene_coords = pd.read_csv(gene_coords_file,sep = '\t')
gene_ref_coords = gene_coords[['GeneID','Symbol','Aliases','start_position_on_the_genomic_accession','end_position_on_the_genomic_accession']]
gene_ref_coords['rang'] = gene_coords.apply(lambda x: (x.start_position_on_the_genomic_accession,
                                                   x.end_position_on_the_genomic_accession),1)
gene_ref_coords.set_index(['rang'],inplace = True)

def look_for_gene(position,gene_coords):
    res = gene_coords[(position> gene_coords.start_position_on_the_genomic_accession) & (position<gene_coords.end_position_on_the_genomic_accession)]
    return(res.Symbol.unique())

start_col = 'start_position_on_the_genomic_accession'
end_col = 'end_position_on_the_genomic_accession'

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [2]:
gen_st =dfrep.st.apply(lambda x:np.unique([look_for_gene(position=x,gene_coords=gene_ref_coords[gene_ref_coords.Symbol!='LMP2'])]))
gen_end =dfrep.end.apply(lambda x:np.unique([look_for_gene(position=x,gene_coords=gene_ref_coords[gene_ref_coords.Symbol!='LMP2'])]))

refs = []
for i in range(len(gen_st)):
    refs.append(np.unique(np.concatenate([gen_st[i],gen_end[i]])))
#refs = pd.concat(refs)

refs2 = [r if len(r)>0 else ['intr'] for r in refs ]
a = []
for i in range(len(refs2)):
    l = len(refs2[i])
    if l==1:
        a.append(pd.Series(i,refs2[i]))
    else:
        a.append(pd.Series([i]*l,refs2[i]))

a = pd.concat(a)
a = a.to_frame().reset_index()
a.columns = ['GENE','indice']

b = pd.merge(dfrep,a,left_index=True,right_on='indice',how = 'left')
rep_with_gene = b[['Mean Rho/bp.','+95% CI','-95% CI','call','GENE']]



In [32]:
b[(b.call=='tandem') &(b['Mean Rho/bp.']>thr)]

,+95% CI,-95% CI,Mean Rho/bp.,call,end,gene,locus_tag,note,rpt_family,rpt_type,rpt_unit_range,st,trimmed_end,trimmed_st,GENE,indice
42,0.04533,0.039273,0.04228,tandem,88043.0,NaN,NaN,NaN,NaN,NaN,NaN,87834.0,89774.0,89562.0,EBNA3B,27
43,0.04533,0.039273,0.04228,tandem,88043.0,NaN,NaN,NaN,NaN,NaN,NaN,87834.0,89774.0,89562.0,EBNA3C,27


In [33]:
b[(b.call=='tandem') &(b.GENE =='LMP1')]

,+95% CI,-95% CI,Mean Rho/bp.,call,end,gene,locus_tag,note,rpt_family,rpt_type,rpt_unit_range,st,trimmed_end,trimmed_st,GENE,indice
53,0.01727,0.01386,0.01551,tandem,168134.0,NaN,NaN,NaN,NaN,NaN,NaN,168036.0,172990.0,172883.0,LMP1,35


In [3]:
def mapp_and_compute_rho(df,start_col = 'st',end_col = 'end'):
    df = df.dropna(subset=[start_col,end_col])
    start_aux = [x if x in mapp_dict.keys() else (x +1 ) for x in df[start_col].values]
    end_aux = [x if x in mapp_dict.keys() else (x -1 ) for x in df[end_col].values]


    df['trimmed_st'] = get_items(mapp_dict,start_aux)
    df['trimmed_end'] = get_items(mapp_dict,end_aux)

    computed_rates = df.apply(lambda x: pd.concat([x,compute_local_obs(rrate,r1 = x.trimmed_st,r2 = x.trimmed_end)]),axis =1)
    return df,computed_rates

def get_items(d,keylist):
    return itemgetter(*keylist)(d)


def search_fasta(pattern, file_path):
    st = []
    end = []
    pat =[]
    reverse = str(Seq(pattern).reverse_complement())
    
    for patt in [pattern,reverse]:
        for record in SeqIO.parse(open(file_path, "rU"), "fasta"):
            chrom = record.id
            for match in re.finditer(patt, str(record.seq)):
                start_pos = match.start() + 1
                end_pos = match.end() + 1
                st.append(start_pos)
                end.append(end_pos)
                pat.append(patt)
    df = pd.DataFrame({'st':st,'end':end,'pattern':pat,'call':pattern})
    df = df[['st','end','pattern','call']]
    df.sort_values(by=['st'],inplace = True)
    
    return(df)


def compute_local_obs(rrate,r1,r2):
    return(rrate[r1:r2].apply(np.mean))

mapfile = '/data/EBV/msas/ids_171_ebv/ids_171_ebv_msa_gap1.51.fa_whole_mapping.file'
outpath = '/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/rdp4_results/ids_171_ebv/10Miter/results/'
genbank_ref_file = '/data/EBV/byACCIDs/genBankRecord_NC_007605.gb'
whole_recom_rate = '/home/ariel/Projects/Gutierrez/EBV-recomb/recomb/rdp4_results/ids_171_ebv/10Miter/whole_10Miters.tsv'


In [4]:
####### map for all downstream analysis####
###########################################
mapp = pd.read_csv(mapfile,sep = '|')
mapp_dict = mapp[['refCoord','trimmedCoord']]

rrate = pd.read_csv(whole_recom_rate,sep = '|')
rrate.set_index(['trimmed_position'],inplace = True)

mapp_dict =  {}
refcoords = mapp.loc[:,'refCoord']
trimcoords = mapp.loc[:,'trimmedCoord']
for i in range(mapp.shape[0]):
    mapp_dict.update({refcoords[i]:trimcoords[i]})


In [5]:
def complete_features(seq_record_genbank, TIPO='regulatory',n=5 ):
    #rates = res3[res3.type == TIPO]
    #rates = rates[~rates['Mean Rho/bp.'].isnull()]

    ttyp = []
    loc = []
    get = []
    pos = []
    for feature in seq_record.features:
        ttyp.append(feature.type)

        if TIPO[0:n] in feature.type:
            get.append({k: feature.qualifiers[k][0] for k in feature.qualifiers.keys()})

            partes = []
            for p in feature.location.parts:
                partes.append(pd.Series([p.start.position,p.end.position]))
            positions = pd.concat(partes,1).transpose()    
            positions.columns = ['start','end']
            pos.append(positions)        #print feature

    get = pd.DataFrame(get)        
    pos = pd.concat(pos)

#    return(get,pos)

    pos.index = get.index
    get_features = pd.concat([get,pos],1)
        
    #rho = rates[['gene','Mean Rho/bp.','start']]#'+95% CI','start','end']]
    #rho = pd.merge(rho,get_features.drop(['gene'],1),on='start',how = 'outer')
    return(get_features)

In [8]:
from Bio import SeqIO
genbank_ref_file = '/data/EBV/byACCIDs/genBankRecord_NC_007605.gb'
seq_record = SeqIO.read(genbank_ref_file,'genbank')

types = []
cds = []
region =[]
for feature in seq_record.features:
    types.append(feature.type)
    if 'gene' in feature.qualifiers.keys():
        gene = feature.qualifiers['gene']
    else:
        gene = 'None'
    
    partes = []
    for p in feature.location.parts:
        partes.append(pd.Series([p.start.position,p.end.position]))
    positions = pd.concat(partes,1).transpose()    
    positions.columns = ['start','end']
    positions['type'] = feature.type
    positions['gene'] = np.repeat(gene,positions.shape[0])

    
    region.append(positions)
regions = pd.concat(region)

regions = regions[regions.type!='source']

# compute length of each region
regions['length'] = regions.end - regions.start
regions['length'] = pd.to_numeric(regions.length)

regions = regions[regions.length < 40000] # con esto tiramos la anomalia del primer gen. 
regions.rename(columns={'gene':'GENE','type':'call','start':'st'},inplace = True)
_, regions_rho = mapp_and_compute_rho(regions)
reg_rho = regions_rho[['call','GENE','Mean Rho/bp.','-95% CI','+95% CI']]

In [9]:
reg_rho = regions_rho[['call','GENE','Mean Rho/bp.','-95% CI','+95% CI']]
bygene = pd.concat([reg_rho,rep_with_gene])
bygene.dropna(subset = ['Mean Rho/bp.'],inplace = True)
result = bygene.groupby(['GENE','call'])['Mean Rho/bp.'].mean()
result = result.reset_index()
result.replace({'intr':'LMP2'},inplace = True)  # PARCHE
result.call.replace('','NC.',inplace = True)
#result.to_csv(outpath+'gene_vs_call.csv')
#result.to_csv(outpath+'gene_vs_call.csv')

In [12]:
thr = rrate.mean()['-95% CI']
genes_ok = result[result['Mean Rho/bp.']>thr]['GENE']
print genes_ok.nunique()
result_above_mean= result[result.GENE.isin(genes_ok)]
result_above_mean.call.replace('','NC.',inplace = True)

#result_above_mean.to_csv(outpath+'filtered_gene_vs_call.csv')

28


/usr/local/lib/python2.7/dist-packages/pandas/core/generic.py:3924: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [14]:
thr = rrate.mean()['-95% CI']
rrep = ['inverted', 'tandem', 'nan', 'type A', 'type B', 'type C', 'NC.','type D','EBNA triplet repeat GGA,GCA,GGG','PstI repeats', 'TERMINAL']
repet = result[result.call.isin(rrep)]
genes_ok = repet[repet['Mean Rho/bp.']>thr]['GENE']
print genes_ok.nunique()

result_above_mean= repet[repet.GENE.isin(genes_ok)]
#result_above_mean.call.replace('','NC.',inplace = True)
#result_above_mean.to_csv(outpath+'filtered_gene_vs_repeat.csv')
result_above_mean

3


,GENE,call,Mean Rho/bp.
176,EBNA3B,tandem,0.021079
181,EBNA3C,tandem,0.028925
222,LMP2,TERMINAL,0.000010
223,LMP2,type A,0.068306
224,LMP2,type B,0.283990
225,LMP2,type C,0.304569
226,LMP2,type D,0.465500


In [28]:
rep_with_gene.head()

,Mean Rho/bp.,+95% CI,-95% CI,call,GENE
0,NaN,NaN,NaN,inverted,intr
1,NaN,NaN,NaN,inverted,intr
2,NaN,NaN,NaN,inverted,intr
3,NaN,NaN,NaN,inverted,intr
4,NaN,NaN,NaN,inverted,BCRF2


In [27]:
regions_rho.call.value_counts()

gene             151
CDS              138
regulatory       125
misc_feature      51
exon              39
repeat_region     39
mRNA              24
intron             4
variation          4
rep_origin         4
misc_RNA           3
ncRNA              3
STS                3
mat_peptide        2
Name: call, dtype: int64